In [1]:
import argparse
import logging
from operator import add
from random import random

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

### Initialize
data_uri = "s3://aws-logs-784082445748-us-east-1/elasticmapreduce/j-29WKE6APA5GV2/final_v2.csv"
spark = SparkSession.builder.appName("work1").getOrCreate()
df = spark.read.options(header="True").csv(data_uri).cache()



VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1623603323085_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
### G1.Q2 #####
dfq2 = df.groupby("UniqueCarrier").agg(F.mean("ArrDelay")) \
        .orderBy("avg(ArrDelay)").select("UniqueCarrier", "avg(ArrDelay)") \
        .take(10)

dfq2

VBox()

Starting Spark application


KeyboardInterrupt: 

In [17]:
def run_g1_3():
    dfq3 = df.groupby("DayOfWeek").agg(F.mean("ArrDelay")) \
             .orderBy("avg(ArrDelay)").select("DayOfWeek", "avg(ArrDelay)")
    return(dfq3)
    
dfq3 = run_g1_3()
dfq3.show()

ans_g1_3 = dfq3.collect()
print(ans_g1_3)

import boto3
client = boto3.client('dynamodb', region_name='us-east-1')

for entry in ans_g1_3:
    print(entry[0], entry[1])
    client.put_item(
        TableName="table_g1_3",
        Item = {
            'DayOfWeek':{"N":entry[0]},
            'AvgDelay': {"S":str(entry[1])}
        })

In [11]:
def run_g2_1():
    dfq1 = df.groupby("Origin", "UniqueCarrier").agg(F.mean("DepDelay"))
    window = Window.partitionBy(dfq1["Origin"]).orderBy(dfq1["avg(DepDelay)"])
    dfq1 = dfq1.select("*", F.rank().over(window).alias("rank")) \
        .filter(F.col("rank")<=10)
    
    dfq1.where(dfq1["Origin"].isin('CMI','BWI','MIA','LAX','IAH','SFO')).show(60)
    
    
run_g2_1()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------+-------------------+----+
|Origin|UniqueCarrier|      avg(DepDelay)|rank|
+------+-------------+-------------------+----+
|   IAH|           NW| 3.5155891221954794|   1|
|   IAH|       PA (1)| 3.9015834348355662|   2|
|   IAH|           PI| 3.9521674140508223|   3|
|   IAH|           US|  5.029792044839336|   4|
|   IAH|           F9|  5.549463301421526|   5|
|   IAH|           AA|  5.616926765989229|   6|
|   IAH|           TW|  6.027507886435331|   7|
|   IAH|           WN|  6.213108817671555|   8|
|   IAH|           OO|  6.583271802444827|   9|
|   IAH|           MQ|  6.709770114942529|  10|
|   CMI|           OH| 0.6120049504950495|   1|
|   CMI|           US|  1.824410452517527|   2|
|   CMI|           TW| 3.7445887445887447|   3|
|   CMI|           PI|  4.369928400954654|   4|
|   CMI|           DH|  6.080160320641283|   5|
|   CMI|           EV|  6.665137614678899|   6|
|   CMI|           MQ| 7.9927815501315225|   7|
|   MIA|           9E|-3.666666666666666

In [86]:
def run_g2_2():
    dfq2 = df.groupby("Origin", "Dest").agg(F.mean("DepDelay"))
    window = Window.partitionBy(dfq2["Origin"]).orderBy(dfq2["avg(DepDelay)"])
    dfq2 = dfq2.select("*", F.rank().over(window).alias("rank")).filter(F.col("rank")<=10)
    
    dfq2.filter(dfq2["Origin"].isin('CMI','BWI','MIA','LAX','IAH','SFO')).show(60)
    
    
run_g2_2()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----+-------------------+----+
|Origin|Dest|      avg(DepDelay)|rank|
+------+----+-------------------+----+
|   IAH| MSN|               -2.0|   1|
|   IAH| MLI|               -1.0|   2|
|   IAH| AGS|-0.6172972972972973|   3|
|   IAH| EFD| 1.8877082136703045|   4|
|   IAH| HOU|  2.172036985149902|   5|
|   IAH| JAC| 2.4226190476190474|   6|
|   IAH| MTJ|   2.90251572327044|   7|
|   IAH| RNO| 3.2272831838049227|   8|
|   IAH| GUC| 3.5373737373737373|   9|
|   IAH| BPT| 3.5995325282430852|  10|
|   CMI| PIT| 1.1078260869565217|   1|
|   CMI| CVG|  1.902127659574468|   2|
|   CMI| DAY| 2.9280490681764566|   3|
|   CMI| PIA|  3.412568306010929|   4|
|   CMI| STL|  3.841726618705036|   5|
|   CMI| DFW|  5.978073610023492|   6|
|   CMI| ATL|  6.665137614678899|   7|
|   CMI| ORD|  8.163503649635036|   8|
|   MIA| SAN|  1.710382513661202|   1|
|   MIA| BUF|                2.0|   2|
|   MIA| SLC| 2.4107883817427385|   3|
|   MIA| HOU| 2.8884911894273126|   4|
|   MIA| ISP| 3.649122807

In [41]:
dfq4 = df.groupby("Origin", "Dest").agg(F.mean("ArrDelay"))
dfq4.filter(F.concat(F.col("Origin"),F.col("Dest")) \
            .isin("CMIORD","INDCMH","DFWIAH","LAXSFO","JFKLAX","ATLPHX")).show()
print(dfq4.count())

import boto3
client = boto3.client('dynamodb', region_name='us-east-1')

for entry in dfq4.rdd.collect():
    client.put_item(
        TableName="table_g2_4",
        Item = {
            "airport":{"S":entry[0]+"-"+entry[1]},
            "avgDelay":{"N":str(entry[2])}
        })

In [2]:
### Q.3.1
dfin = df.groupby("Origin").count()
dfout = df.groupby("Dest").count()

pair = (dfin.rdd+dfout.rdd).reduceByKey(add).collect()
value = [v for (k,v) in pair]
print(value)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[6956, 65156, 45200, 2160830, 3963056, 1515269, 204561, 1533418, 110463, 1165, 72614, 321190, 136564, 3555, 99982, 12438, 36688, 4724, 52329, 158176, 6494512, 120032, 8768, 32669, 388405, 50960, 34061, 3332, 611758, 16167, 224, 67750, 38376, 821, 48752, 49638, 121982, 52314, 120865, 36861, 1491, 29376, 3359, 62005, 2479807, 117564, 373034, 313077, 1283, 7264, 32867, 500415, 51114, 13233, 704929, 811805, 17424, 60654, 10687, 2450530, 88098, 3175300, 1767339, 274373, 1281, 33543, 538, 11301229, 176, 1077988, 18015, 11830, 13870, 96293, 271666, 13462, 126129, 31693, 90544, 90389, 190765, 18150, 77107, 10953, 7916, 215161, 50482, 658168, 14, 36256, 1032, 128879, 2655993, 140268, 30797, 1416068, 11757, 19964, 7574328, 223629, 3345746, 291234, 16882, 754529, 970, 619803, 33799, 223237, 821791, 539578, 37238, 4909631, 3849311, 255538, 238924, 412661, 82780, 10796, 30983, 1756, 29358, 8614, 234963, 17683, 228782, 24177, 10292, 86755, 286274, 5050872, 1, 445222, 4726736, 107524, 144033, 549, 24

In [32]:
### Q.3.2 ####
df1 = df.filter((F.lpad(df["CRSDepTime"],4,"0")<"1200") & (F.col("FlightDate").substr(1,4)=="2008")) \
    .select(df["Origin"], df["Dest"], F.concat(F.col('UniqueCarrier'),F.lit(" "),F.col('FlightNum')).alias("Flight"), \
            df["ArrDelay"], F.to_timestamp(F.concat(F.col("FlightDate"),F.lpad(df["CRSDepTime"],4,"0")), "yyyy-MM-ddHHmm").alias("CRSDep"), \
            F.to_date(F.col("FlightDate"), "yyyy-MM-dd").alias("Date"))
            
window = Window.partitionBy(df1["Origin"],df1["Dest"],df1["Date"]).orderBy(F.col("ArrDelay").cast("float"))
df1x = df1.select("*", F.rank().over(window).alias("rank")).filter(F.col("rank")<=1)

for cname in df1x.columns:
     df1x = df1x.withColumnRenamed(cname,"l_"+cname)
        

        
df2 = df.filter((F.lpad(df["CRSDepTime"],4,"0")>"1200") & (F.col("FlightDate").substr(1,4)=="2008")) \
    .select(df["Origin"], df["Dest"], F.concat(F.col('UniqueCarrier'),F.lit(" "),F.col('FlightNum')).alias("Flight"), \
            df["ArrDelay"], F.to_timestamp(F.concat(F.col("FlightDate"),F.lpad(df["CRSDepTime"],4,"0")), "yyyy-MM-ddHHmm").alias("CRSDep"), \
            F.to_date(F.col("FlightDate"), "yyyy-MM-dd").alias("Date"))
window = Window.partitionBy(df2["Origin"],df2["Dest"],df2["Date"]).orderBy(F.col("ArrDelay").cast("float"))
df2x = df2.select("*", F.rank().over(window).alias("rank")).filter(F.col("rank")<=1)

for cname in df2x.columns:
     df2x = df2x.withColumnRenamed(cname,"r_"+cname)

cond = [F.col("l_Dest")==F.col("r_Origin"), F.datediff(F.col("r_CRSDep"), F.col("l_CRSDep"))==2]
dfjoin = df1x.join(df2x, cond)

df3 = dfjoin.select("l_Origin", "l_Dest","l_Flight", \
                      F.date_format(F.col("l_CRSDep"),"HH:mm dd/MM/yyyy").alias("l_CRSDep"), \
                      "l_ArrDelay", "r_Origin", "r_Dest","r_Flight", \
                      F.date_format(F.col("r_CRSDep"),"HH:mm dd/MM/yyyy").alias("r_CRSDep"), \
                      "r_ArrDelay", \
                     (F.col("l_ArrDelay")+F.col("r_ArrDelay")).alias("TotDelay"))

In [39]:
df3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|l_Origin|l_Dest|l_Flight|        l_CRSDep|l_ArrDelay|r_Origin|r_Dest|r_Flight|        r_CRSDep|r_ArrDelay|TotDelay|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|     ANC|   DLG|  AS 164|09:44 24/08/2008|     -18.0|     DLG|   ANC|  AS 162|17:50 26/08/2008|      65.0|    47.0|
|     AUS|   MSY|  XE 519|07:15 22/01/2008|      -5.0|     MSY|   DFW| AA 1153|15:10 24/01/2008|      -9.0|   -14.0|
|     AUS|   MSY|  XE 519|07:15 22/01/2008|      -5.0|     MSY|   RDU|  XE 320|17:45 24/01/2008|      50.0|    45.0|
|     AUS|   MSY|  XE 519|07:15 22/01/2008|      -5.0|     MSY|   CLT| US 1634|18:25 24/01/2008|      20.0|    15.0|
|     AUS|   MSY|  XE 519|07:15 22/01/2008|      -5.0|     MSY|   EWR|  CO 217|14:50 24/01/2008|      -7.0|   -12.0|
|     AUS|   MSY|  XE 519|07:15 22/01/2008|      -5.0|     MSY| 

In [38]:
df3.where("l_Origin=='CMI' and l_Dest=='ORD' and r_Dest=='LAX' and l_CRSDep LIKE '%04/03/2008%'").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|l_Origin|l_Dest|l_Flight|        l_CRSDep|l_ArrDelay|r_Origin|r_Dest|r_Flight|        r_CRSDep|r_ArrDelay|TotDelay|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|     CMI|   ORD| MQ 4278|07:10 04/03/2008|     -14.0|     ORD|   LAX|  AA 607|19:50 06/03/2008|     -24.0|   -38.0|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+

In [29]:
df3.where("l_Flight=='MQ 4278' and l_CRSDep=='07:10 04/03/2008' and r_Flight=='AA 607'").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------+--------+----------+--------+------+--------+--------+----------+--------+
|l_Origin|l_Dest|l_Flight|l_CRSDep|l_ArrDelay|r_Origin|r_Dest|r_Flight|r_CRSDep|r_ArrDelay|TotDelay|
+--------+------+--------+--------+----------+--------+------+--------+--------+----------+--------+
+--------+------+--------+--------+----------+--------+------+--------+--------+----------+--------+

In [4]:
df3.where("l_Flight=='AA 845' and l_CRSDep=='07:25 09/09/2008' and r_Flight=='MQ 3627'").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|l_Origin|l_Dest|l_Flight|        l_CRSDep|l_ArrDelay|r_Origin|r_Dest|r_Flight|        r_CRSDep|r_ArrDelay|TotDelay|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|     JAX|   DFW|  AA 845|07:25 09/09/2008|       1.0|     DFW|   CRP| MQ 3627|16:45 11/09/2008|      -7.0|    -6.0|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+

In [5]:
df3.where("l_Flight=='OO 3755' and l_CRSDep=='11:00 01/04/2008' and r_Flight=='OO 5429'").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|l_Origin|l_Dest|l_Flight|        l_CRSDep|l_ArrDelay|r_Origin|r_Dest|r_Flight|        r_CRSDep|r_ArrDelay|TotDelay|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|     SLC|   BFL| OO 3755|11:00 01/04/2008|      12.0|     BFL|   LAX| OO 5429|14:55 03/04/2008|       6.0|    18.0|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+

In [6]:
df3.where("l_Flight=='WN 3534' and l_CRSDep=='06:50 12/07/2008' and r_Flight=='US 412'").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|l_Origin|l_Dest|l_Flight|        l_CRSDep|l_ArrDelay|r_Origin|r_Dest|r_Flight|        r_CRSDep|r_ArrDelay|TotDelay|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|     LAX|   SFO| WN 3534|06:50 12/07/2008|     -13.0|     SFO|   PHX|  US 412|19:25 14/07/2008|     -19.0|   -32.0|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+

In [11]:
df3.where("l_Origin='DFW' and l_CRSDep=='07:00 10/06/2008' and r_Flight=='AA 2341'").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|l_Origin|l_Dest|l_Flight|        l_CRSDep|l_ArrDelay|r_Origin|r_Dest|r_Flight|        r_CRSDep|r_ArrDelay|TotDelay|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|     DFW|   ORD| UA 1104|07:00 10/06/2008|     -21.0|     ORD|   DFW| AA 2341|16:45 12/06/2008|     -10.0|   -31.0|
|     DFW|   ORD| AA 2324|07:00 10/06/2008|       6.0|     ORD|   DFW| AA 2341|16:45 12/06/2008|     -10.0|    -4.0|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+

In [8]:
df3.where("l_Flight=='UA 944' and l_CRSDep=='07:05 01/01/2008' and r_Flight=='B6 918'").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|l_Origin|l_Dest|l_Flight|        l_CRSDep|l_ArrDelay|r_Origin|r_Dest|r_Flight|        r_CRSDep|r_ArrDelay|TotDelay|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|     LAX|   ORD|  UA 944|07:05 01/01/2008|       1.0|     ORD|   JFK|  B6 918|19:00 03/01/2008|      -7.0|    -6.0|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+

In [14]:
print("""
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|l_Origin|l_Dest|l_Flight|        l_CRSDep|l_ArrDelay|r_Origin|r_Dest|r_Flight|        r_CRSDep|r_ArrDelay|TotDelay|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|     CMI|   ORD| MQ 4278|07:10 04/03/2008|     -14.0|     ORD|   LAX|  AA 607|19:50 06/03/2008|     -24.0|   -38.0|
|     JAX|   DFW|  AA 845|07:25 09/09/2008|       1.0|     DFW|   CRP| MQ 3627|16:45 11/09/2008|      -7.0|    -6.0|
|     SLC|   BFL| OO 3755|11:00 01/04/2008|      12.0|     BFL|   LAX| OO 5429|14:55 03/04/2008|       6.0|    18.0|
|     LAX|   SFO| WN 3534|06:50 12/07/2008|     -13.0|     SFO|   PHX|  US 412|19:25 14/07/2008|     -19.0|   -32.0|
|     DFW|   ORD| UA 1104|07:00 10/06/2008|     -21.0|     ORD|   DFW| AA 2341|16:45 12/06/2008|     -10.0|   -31.0|
|     LAX|   ORD|  UA 944|07:05 01/01/2008|       1.0|     ORD|   JFK|  B6 918|19:00 03/01/2008|      -7.0|    -6.0|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|l_Origin|l_Dest|l_Flight|        l_CRSDep|l_ArrDelay|r_Origin|r_Dest|r_Flight|        r_CRSDep|r_ArrDelay|TotDelay|
+--------+------+--------+----------------+----------+--------+------+--------+----------------+----------+--------+
|     CMI|   ORD| MQ 4278|07:10 04/03/2008|     -14.0|     ORD|   LAX|  AA 607|19:50 06/03/2008|     -24.0|   -38.0|
|     JAX|   DFW|  AA 845|07:25 09/09/2008|       1.0|     DFW|   CRP| MQ 3627|16:45 11/09/2008|      -7.0|    -6.0|
|     SLC|   BFL| OO 3755|11:00 01/04/2008|      12.0|     BFL|   LAX| OO 5429|14:55 03/04/2008|       6.0|    18.0|
|     LAX|   SFO| WN 3534|06:50 12/07/2008|     -13.0|     SFO|   PHX|  US 412|19:25 14/07/2008|     -19.0|   -32.0|
|     DFW|   ORD| UA 1104|07:00 10/06/2008|     -21.0|     ORD|   DFW| AA 2341|16:45 12/06/2008|     -10.0|   -31.0|
|     LAX|   ORD|  UA 944|07:05 01/01/2008|       1.0|     ORD|

In [18]:
dfres.where("(l_Flight=='MQ 4278' and l_CRSDep=='07:10 04/03/2008' and r_Flight=='AA 607') or \
            (l_Flight=='AA 845' and l_CRSDep=='07:25 09/09/2008' and r_Flight=='MQ 3627') or \
            (l_Flight=='OO 3755' and l_CRSDep=='11:00 01/04/2008' and r_Flight=='OO 5429') or \
            (l_Flight=='WN 3534' and l_CRSDep=='06:50 12/07/2008' and r_Flight=='US 412') or \
            (l_Flight=='UA 1104' and l_CRSDep=='07:00 10/06/2008' and r_Flight=='AA 2341') or \
            (l_Flight=='UA 944' and l_CRSDep=='07:05 01/01/2008' and r_Flight=='B6 918') or \
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 